In [ ]:
import os

## import pyspark
import pyspark
from pyspark.sql import SparkSession

In [ ]:
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.3.1,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.spark_catalog','org.apache.iceberg.spark.SparkSessionCatalog')
        .set('spark.sql.catalog.spark_catalog.type','hive')
        .set('spark.sql.catalog.iceberg', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.iceberg.type', 'hadoop')
        .set('spark.sql.catalog.iceberg.warehouse', 'iceberg-warehouse')
)

In [ ]:
## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

# Copy On Write

In [ ]:
spark.sql(''' CREATE TABLE iceberg.db.students (
    id int,
    first_name string,
    last_name string,
    major string,
    class_year int
) USING iceberg 
 TBLPROPERTIES (
    'write.delete.mode'='copy-on-write',
    'write.update.mode'='copy-on-write',
    'write.merge.mode'='copy-on-write'
)
PARTITIONED BY (class_year) ;''')

In [ ]:
spark.sql('''
INSERT INTO iceberg.db.students (id, first_name, last_name, major, class_year)
VALUES
(1, 'John', 'Doe', 'Computer Science', 2023),
(2, 'Jane', 'Doe', 'Business', 2019),
(3, 'Peter', 'Smith', 'Engineering', 2021),
(4, 'Susan', 'Williams', 'Nursing', 2023),
(5, 'David', 'Johnson', 'Law', 2020),
(6, 'Elizabeth', 'Brown', 'Art', 2021),
(7, 'Michael', 'Green', 'History', 2019),
(8, 'Sarah', 'White', 'English', 2020),
(9, 'William', 'Black', 'Mathematics', 2021),
(10, 'Mary', 'Brown', 'Physics', 2022),
(11, 'Thomas', 'Green', 'Chemistry', 2022),
(12, 'Jennifer', 'White', 'Biology', 2020),
(13, 'Charles', 'Black', 'Geology', 2020),
(14, 'Lisa', 'Brown', 'Astronomy', 2019),
(15, 'Henry', 'Green', 'Meteorology', 2020),
(16, 'Nancy', 'White', 'Economics', 2022),
(17, 'Daniel', 'Black', 'Political Science', 2022),
(18, 'Emily', 'Brown', 'Philosophy', 2021),
(19, 'Matthew', 'Green', 'Psychology', 2019),
(20, 'Jessica', 'White', 'Sociology', 2021); ''')

In [ ]:
#Reading 
spark.sql("select * from iceberg.db.students").show()

In [ ]:
#Updating
spark.sql("update iceberg.db.students set major = 'zoology' where id = 4;")

In [ ]:
spark.sql("select * from iceberg.db.students").show()

In [ ]:
#deleting records
spark.sql("delete from iceberg.db.students where class_year < 2020;")

In [ ]:
spark.sql("select * from iceberg.db.students").show()

In [ ]:
#Creating a new table used to merge records
spark.sql(''' CREATE TABLE iceberg.db.new_table(
    id int,
    first_name string,
    last_name string,
    major string,
    class_year int
) USING iceberg 
''')

In [ ]:
spark.sql('''
INSERT INTO iceberg.db.new_table(id, first_name, last_name, major, class_year)
VALUES
(1, 'James', 'smith', 'Commerce', 2023),
(2, 'Jane', 'Foster', 'Astrology', 2018),
(20, 'Peter', 'Parker', 'Aerospace', 2019); ''')

In [ ]:
#merging records from new_table into students

In [ ]:
spark.sql('''Merge into iceberg.db.students
using (select * from iceberg.db.new_table) nt 
on iceberg.db.students.id = nt.id
when matched then update set 
iceberg.db.students.id = nt.id,
iceberg.db.students.first_name = nt.first_name,
iceberg.db.students.last_name = nt.last_name,
iceberg.db.students.major = nt.major,
iceberg.db.students.class_year = nt.class_year
when not matched then insert *;
''')

In [ ]:
spark.sql("select * from iceberg.db.students").show()

In [ ]:
#inspecting table history
df=spark.sql("SELECT * FROM iceberg.db.students.history")
df.show()
df.write.save("/Users/anshumanr/Documents/Iceberg/Apache_iceberg/iceberg-warehouse/db/students/history",header=True)

In [ ]:
#inspecting table history
df=spark.sql("SELECT * FROM iceberg.db.students.snapshots")
df.show()
df.write.save("/Users/anshumanr/Documents/Iceberg/Apache_iceberg/iceberg-warehouse/db/students/snapshots",header=True)

# Merge On Read

In [ ]:
spark.sql(''' CREATE TABLE iceberg.db.students2 (
    id int,
    first_name string,
    last_name string,
    major string,
    class_year int
) USING iceberg 
 TBLPROPERTIES (
    'write.delete.mode'='merge-on-read',
    'write.update.mode'='merge-on-read',
    'write.merge.mode'='merge-on-read'
)
PARTITIONED BY (class_year) ;''')

In [ ]:
spark.sql('''
INSERT INTO iceberg.db.students2 (id, first_name, last_name, major, class_year)
VALUES
(1, 'John', 'Doe', 'Computer Science', 2023),
(2, 'Jane', 'Doe', 'Business', 2019),
(3, 'Peter', 'Smith', 'Engineering', 2021),
(4, 'Susan', 'Williams', 'Nursing', 2023),
(5, 'David', 'Johnson', 'Law', 2020),
(6, 'Elizabeth', 'Brown', 'Art', 2021),
(7, 'Michael', 'Green', 'History', 2019),
(8, 'Sarah', 'White', 'English', 2020),
(9, 'William', 'Black', 'Mathematics', 2021),
(10, 'Mary', 'Brown', 'Physics', 2022),
(11, 'Thomas', 'Green', 'Chemistry', 2022),
(12, 'Jennifer', 'White', 'Biology', 2020),
(13, 'Charles', 'Black', 'Geology', 2020),
(14, 'Lisa', 'Brown', 'Astronomy', 2019),
(15, 'Henry', 'Green', 'Meteorology', 2020),
(16, 'Nancy', 'White', 'Economics', 2022),
(17, 'Daniel', 'Black', 'Political Science', 2022),
(18, 'Emily', 'Brown', 'Philosophy', 2021),
(19, 'Matthew', 'Green', 'Psychology', 2019),
(20, 'Jessica', 'White', 'Sociology', 2021); ''')